# Long Short-Term Memory (LSTM)

This page attempts to explain why LSTM was first proposed, and what are the core features together with some examples.

This is based on the paper [Hochreiter and Schmidhuber. 1997. Long Short-Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)

## Constant-Error Carrousel

The core feature of an LSTM unit as first proposed is the constant-error carrousel (CEC) which solves the vanishing gradient problem with standard RNN.

A CEC is neural network unit which consists of a single neuron with self-loop with weight fixed to 1.0 to ensure constant error flow when doing backpropagation.

<div style="text-align:center">
<img src="CEC.png"/>
<caption>Fig 1. Diagram of a single CEC unit</caption>
</div>

Now let's see an example of CEC at work. We will use CEC to do very simple task: **recognizing whether current character is inside a bracketed expression**, with the opening bracket considered to be inside, and the closing bracket considered to be outside, for simplicity. This is solvable only using network that can store memory, since to recognize whether a character is inside a bracketed expression, we need to have the knowledge that there is an opening bracket to the left of current character which does not have the corresponding closing bracket.

The input alphabets are coming from the set: $\{`a`, `b`, `(`, `)`\}$ with the following 2-dimensional embedding:

$$
\begin{eqnarray}
emb(`a`) &=& (1, 1) \nonumber\\
emb(`b`) &=& (-1, -1) \nonumber\\
emb(`(`) &=& (1, 0) \nonumber\\
emb(`)`) &=& (0, -1) \nonumber
\end{eqnarray}
$$

For this task, we define a very simple network with two input units, one CEC unit, and one output unit with sigmoid activation ($\sigma(x) = \frac{1}{1 + e^{-x}}$), as follows:

<div style="text-align:center">
![Fig 2. Network used for the bracketed expression recognition](CEC-example.png)
<caption>Fig 2. Network used for the bracketed expression recognition</caption>
</div>

For this task, we define the loss function as the cross-entropy (CE) between the predicted and the true one:

$$
\begin{eqnarray}
\mathrm{CE}(x, y) = - (x\log(y) + (1-x)\log(1-y)) \nonumber\\
\mathrm{Loss}(\hat{o}_t, o_t) = \mathrm{CE}(\hat{o}_t, o_t) - \mathrm{CE}(\hat{o}_t, \hat{o}_t)
\end{eqnarray}
$$

with $\hat{o}_t$ and $o_t$ represent the target value (gold standard) and output value (network prediction), respectively, at time step $t$. The first term is the cross-entropy between the target value and the output value, and the second term is the entropy of the target value itself. Note that the second term is a constant, and serves just to make the minimum achievable loss to be 0 (perfect output).

More specifically, we have:

$$
\begin{equation}
o_t = \sigma(w_3*s_t)
\end{equation}
$$

where $s_t$ is the output of the CEC unit (a.k.a. the memory), which depends on the previous value of the memory $s_{t-1}$, and the input $x_{t,1}$ and $x_{t,2}$ (representing the first and second dimension of the input at time step $t$):

$$
\begin{equation}
s_t = \underbrace{w_s * s_{t-1}}_\text{previous value} + \underbrace{w_1 * x_{t,1} + w_2 * x_{t,2}}_\text{input}
\end{equation}
$$

where $w_s$ is the weight of the self-loop, which is 1.0. But for clarity of why this should be 1.0, the calculation later does not assume $w_s=1.0$.

In [1]:
import math
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# Embedding
embedding = {}
embedding['a'] = (1, 1)
embedding['b'] = (-1, -1)
embedding['('] = (1, 0)
embedding[')'] = (0, 1)

# embedding['a'] = (-1, 0)
# embedding['b'] = (-0.5, 0)
# embedding['('] = (1, 1)
# embedding[')'] = (1, -1)

# Weights
w1=1.0
w2=1.0
w3=1.0
ws=1.0

memory_history = [0]
output_history = [0]

def sigmoid(x):
    return 1.0/(1+math.exp(-x))

def gold(seq):
    result = [0]
    in_bracket = False
    for char in seq:
        if char == '(':
            in_bracket = True
        if char == ')':
            in_bracket = False
        if in_bracket:
            result.append(sigmoid(1))
        else:
            result.append(sigmoid(0))
    return result

def activate_memory(x1, x2):
    prev_memory = memory_history[-1]
    memory_history.append(ws*prev_memory + w1*x1 + w2*x2)
    return memory_history[-1]

def activate_output(h):
    output_history.append(sigmoid(w3*h))
    return output_history[-1]

def predict(seq):
    for char in seq:
        activate_output(activate_memory(*embedding[char]))
    result = output_history[:]
    return result

def reset():
    global memory_history, output_history
    memory_history = [0]
    output_history = [0]
    
def loss(gold_seq, pred_seq):
    result = 0.0
    per_position_loss = []
    for idx, (corr, pred) in enumerate(zip(gold_seq, pred_seq)):
        cur_loss  = -(corr*math.log(pred) + (1-corr)*math.log(1-pred))
        cur_loss -= -(corr*math.log(corr) + (1-corr)*math.log(1-corr))
        result += cur_loss
        per_position_loss.append(cur_loss)
    return result, per_position_loss


def print_list(lst):
    '''A convenience method to print a list of real numbers'''
    as_str = ['{:+.3f}'.format(num) for num in lst]
    print('[{}]'.format(', '.join(as_str)))

In [2]:
# See typical values of sigmoid
for i in range(5):
    print('sigmoid({}) = {}'.format(i, sigmoid(i)))

sigmoid(0) = 0.5
sigmoid(1) = 0.7310585786300049
sigmoid(2) = 0.8807970779778823
sigmoid(3) = 0.9525741268224334
sigmoid(4) = 0.9820137900379085


Now let's check the function calculating the target value. Basically we want it to output $\sigma(0)$ or $\sigma(1)$ when the output is outside or inside a bracketed expression, respectively.

In [3]:
gold('a(a)a')[1:]  # The first element is dummy

[0.5, 0.7310585786300049, 0.7310585786300049, 0.5, 0.5]

Which is $\sigma(0), \sigma(1), \sigma(1), \sigma(0), \sigma(0)$, which is what we expect. So far so good.
<hr>
Now let's see what our network outputs

In [4]:
test_seq = 'ab(ab)ab'
reset()
w1 = 1.0
w2 = 1.0
w3 = 1.0
result = predict(test_seq)
correct = gold(test_seq)
print('Output: ', end='')
print_list(result[1:])
print('Target: ', end='')
print_list(correct[1:])
print('Loss  : {:.3f}'.format(loss(correct[1:], result[1:])[0]))

Output: [+0.881, +0.500, +0.731, +0.953, +0.731, +0.881, +0.982, +0.881]
Target: [+0.500, +0.500, +0.731, +0.731, +0.731, +0.500, +0.500, +0.500]
Loss  : 2.900


We see that the loss is still non-zero, and we see that some values are incorrectly predicted.

Next we will see the gradient calculation in progress, so that we can update the weight to reduce the loss.

### Calculating Gradients

To do the weight update, we need to calculate the partial derivative of the loss function with respect to the each weight. We have three weight parameters $w_1, w_2$, and $w_3$, so we need to compute three different partial derivatives.

For ease of notation, we denote $\mathrm{Loss}_t = \mathrm{Loss}(\hat{o}_t, o_t)$ as the loss at time step $t$ and $\mathrm{Loss} = \sum_t \mathrm{Loss}_t$ as the total loss over one sequence.

Remember that our objective is to reduce the total loss.

$$
\begin{eqnarray}
\frac{\partial\mathrm{Loss}}{\partial w_i} & = & \sum_t\frac{\partial \mathrm{Loss}_t}{\partial w_i} \\
& = & \sum_t\frac{\partial \mathrm{Loss}_t}{\partial o_t} \cdot \frac{\partial o_t}{\partial w_i} \qquad \text{(by chain rule)} \\
\end{eqnarray}
$$

for $w_3$, we can already compute the gradient here, which is:

$$
\require{cancel}
\begin{eqnarray}
\frac{\partial\mathrm{Loss}}{\partial w_3} & = & \sum_t\frac{\partial \mathrm{Loss}_t}{\partial o_t} \cdot \frac{\partial o_t}{\partial w_i} \\
& = & \sum_t\underbrace{\frac{o_t - \hat{o}_t}{\cancel{o_t(1-o_t)}}}_{=\frac{\partial \mathrm{Loss}_t}{\partial o_t}} \cdot \underbrace{s_t \cdot \cancel{o_t(1-o_t)}}_{=\frac{\partial o_t}{\partial w_i}} \\
& = & \sum_t(o_t-\hat{o}_t)s_t
\end{eqnarray}
$$

for $w_1$ and $w_2$, we have:

$$
\begin{eqnarray}
\frac{\partial\mathrm{Loss}}{\partial w_i} & = & \sum_t\frac{\partial \mathrm{Loss}_t}{\partial o_t} \cdot \frac{\partial o_t}{\partial w_i} \\
& = & \sum_t \frac{o_t - \hat{o}_t}{o_t(1-o_t)} \cdot \frac{\partial o_t}{\partial s_t} \cdot \frac{\partial s_t}{\partial w_i} \\
& = & \sum_t \frac{o_t - \hat{o}_t}{\cancel{o_t(1-o_t)}} \cdot w_3\cdot \cancel{o_t(1-o_t)} \cdot \frac{\partial s_t}{\partial w_i} \\
& = & \sum_t (o_t - \hat{o}_t)w_3 \cdot \frac{\partial s_t}{\partial w_i} \\
& = & \sum_t (o_t - \hat{o}_t)w_3 \cdot \left(w_s\cdot\frac{\partial s_{t-1}}{\partial w_i} + x_{t,i}\right) \\
& = & \sum_t (o_t - \hat{o}_t)w_3 \cdot \left({w_s}^2\cdot\frac{\partial s_{t-2}}{\partial w_i} + w_s\cdot x_{t-1,i} + x_{t,i}\right) \\
& & \ldots \\
& = & \sum_t (o_t - \hat{o}_t)w_3 \cdot \left(\sum_{t'\leq t} {w_s}^{t-t'}x_{t',i}\right) \\
\end{eqnarray}
$$

### Important Note on $w_s$!

We see that the gradient with respect to $w_1$ and $w_2$ contains the factor ${w_s}^{t-t'}$, where $t-t'$ can be as large as the input sequence length. So if $w_s \neq 1.0$, then either the gradient will vanish or blow up as the input sequence gets longer.

In [5]:
def dLdw1(test_seq, gold_seq, pred_seq, state_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw1 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(ws**(step-1)*embedding[test_seq[step-1]][0] for step in range(1, time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def dLdw2(test_seq, gold_seq, pred_seq, state_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw2 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(ws**(step-1)*embedding[test_seq[step-1]][1] for step in range(1, time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def dLdw3(test_seq, gold_seq, pred_seq, state_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw3 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * state_seq[time_step]
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

## Experiment
Now we define an experiment which takes in initial values of all the weights, learning rate, and maximum number of iterations. We also want to experiment with fixing the weight $w_3$ (i.e., it is not learned).

The code below will print the total loss, the loss at each time step, the output, target, and memory at each time step, and also the gradient for each learned parameter at each time step.

In [6]:
def experiment(test_seq, _w1=1.0, _w2=1.0, _w3=1.0, alpha=1e-1, max_iter=250, fixed_w3=True):
    global w1, w2, w3
    reset()
    w1 = _w1
    w2 = _w2
    w3 = _w3
    correct = gold(test_seq)
    print('w1={:+.3f}, w2={:+.3f}, w3={:+.3f}'.format(w1, w2, w3))
    for iter_num in range(max_iter):
        result = predict(test_seq)
        printmd('**Iteration {}**'.format(iter_num))
        print('Output: ', end='')
        print_list(result)
        print('Target: ', end='')
        print_list(correct)
        print('Memory: ', end='')
        print_list(memory_history)
        total_loss, per_position_loss = loss(correct[1:], result[1:])
        loss_str = 'Loss: **{:.5f}** = <span style="font-family:monaco; font-size:12px">'.format(total_loss)
        for idx, per_pos_loss in enumerate(per_position_loss):
            loss_str += '{}{:.3f}'.format(' + ' if idx > 0 else '', per_pos_loss)
        loss_str += '</span>'
        printmd(loss_str)
        w1 -= alpha * dLdw1(test_seq, correct, result, memory_history)
        w2 -= alpha * dLdw2(test_seq, correct, result, memory_history)
        if not fixed_w3:
            w3 -= alpha * dLdw3(test_seq, correct, result, memory_history)
        print('w1={:+.3f}, w2={:+.3f}, w3={:+.3f}'.format(w1, w2, w3))
        print()
        reset()
    return w1, w2, w3

In [17]:
w1, w2, w3 = experiment('ab(ab)ab', alpha=1e-1, fixed_w3=False)
experiment('aabba(a(b)a)bab', _w1=w1, _w2=w2, _w3=w3, max_iter=1)

w1=+1.000, w2=+1.000, w3=+1.000


**Iteration 0**

Output: [+0.000, +0.881, +0.500, +0.731, +0.953, +0.731, +0.881, +0.982, +0.881]
Target: [+0.000, +0.500, +0.500, +0.731, +0.731, +0.731, +0.500, +0.500, +0.500]
Memory: [+0.000, +2.000, +0.000, +1.000, +3.000, +1.000, +2.000, +4.000, +2.000]


Loss: **2.89955** = <span style="font-family:monaco; font-size:12px">0.434 + 0.000 + 0.000 + 0.273 + 0.000 + 0.434 + 1.325 + 0.434</span>

<span style="font-family:monaco; font-size:12px">dL/dw1 = <span style="color:blue">+0.381</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.443</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.381</span> + <span style="color:blue">+0.964</span> + <span style="color:blue">+0.381</span> = <span style="color:blue; text-decoration:underline">+2.549</span></span>

...

Loss: **0.40526** = <span style="font-family:monaco; font-size:12px">0.000 + 0.000 + 0.000 + 0.000 + 0.000 + 0.000 + 0.000 + 0.083 + 0.083 + 0.120 + 0.120 + 0.000 + 0.000 + 0.000 + 0.000</span>

<span style="font-family:monaco; font-size:12px">dL/dw1 = <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:red">-0.000</span> + <span style="color:red">-0.000</span> + <span style="color:blue">+0.612</span> + <span style="color:blue">+0.459</span> + <span style="color:blue">+0.709</span> + <span style="color:blue">+0.945</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> = <span style="color:blue; text-decoration:underline">+2.727</span></span>

<span style="font-family:monaco; font-size:12px">dL/dw2 = <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:red">-0.000</span> + <span style="color:red">-0.000</span> + <span style="color:blue">+0.306</span> + <span style="color:blue">+0.153</span> + <span style="color:blue">+0.473</span> + <span style="color:blue">+0.709</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> + <span style="color:blue">+0.000</span> = <span style="color:blue; text-decoration:underline">+1.642</span></span>

w1=+0.705, w2=-1.142, w3=+1.023



(0.7047403857313423, -1.1415477624522203, 1.0227737600906985)

# Let's Try Adding Input Gate

In [13]:
w4 = 1.0
w5 = 1.0
input_history = [0]
gate_history = [0]

def reset_gated():
    global memory_history, output_history, input_history, gate_history
    memory_history = [0]
    output_history = [0]
    input_history = [0]
    gate_history = [0]

def activate_input(x1, x2):
    result = (w1*x1+w2*x2)
    input_history.append(result)
    return result

def activate_gate(x1, x2):
    result = sigmoid(w4*x1+w5*x2)
    gate_history.append(result)
    return result

def dLdw1_gated(test_seq, gold_seq, pred_seq, state_seq, input_seq, gate_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw1 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(embedding[test_seq[step-1]][0]*gate_seq[step] for step in range(1, time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def dLdw2_gated(test_seq, gold_seq, pred_seq, state_seq, input_seq, gate_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw2 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(embedding[test_seq[step-1]][1]*gate_seq[step] for step in range(1, time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def dLdw4_gated(test_seq, gold_seq, pred_seq, state_seq, input_seq, gate_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw4 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(embedding[test_seq[step-1]][0]*gate_seq[step]*input_seq[step]*(1-gate_seq[step]) for step in range(1,time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def dLdw5_gated(test_seq, gold_seq, pred_seq, state_seq, input_seq, gate_seq):
    result = 0.0
    grad_str = '<span style="font-family:monaco; font-size:12px">dL/dw5 = '
    for time_step in range(1, len(gold_seq)):
        cur_dell = (pred_seq[time_step] - gold_seq[time_step]) * w3
        cur_dell *= sum(embedding[test_seq[step-1]][1]*gate_seq[step]*input_seq[step]*(1-gate_seq[step]) for step in range(1,time_step+1))
        if cur_dell < 0:
            color = 'red'
        else:
            color = 'blue'
        grad_str += '{}<span style="color:{}">{:+.3f}</span>'.format(' + ' if time_step > 1 else '', color, cur_dell)
        result += cur_dell
    grad_str += ' = <span style="color:{}; text-decoration:underline">{:+.3f}</span></span>'.format('red' if result < 0 else 'blue', result)
    printmd(grad_str)
    return result

def activate_memory_gated(x1, x2):
    memory_history.append(ws*memory_history[-1] + input_history[-1]*gate_history[-1])
    return memory_history[-1]

def predict_gated(seq):
    for char in seq:
        activate_input(*embedding[char])
        activate_gate(*embedding[char])
        activate_output(activate_memory_gated(*embedding[char]))
    result = output_history[:]
    return result

def experiment_gated(test_seq, _w1=1.0, _w2=1.0, _w3=1.0, _w4=1.0, _w5=1.0, alpha=1e-1, max_iter=750, fixed_w3=True):
    global w1, w2, w3, w4, w5
    reset_gated()
    w1 = _w1
    w2 = _w2
    w3 = _w3
    w4 = _w4
    w5 = _w5
    correct = gold(test_seq)
    print('w1={:+.3f}, w2={:+.3f}, w3={:+.3f}, w4={:+.3f}, w5={:+.3f}'.format(w1, w2, w3, w4, w5))
    for iter_num in range(max_iter):
        result = predict_gated(test_seq)
        printmd('**Iteration {}**'.format(iter_num))
        print('Output: ', end='')
        print_list(result)
        print('Target: ', end='')
        print_list(correct)
        print('Memory: ', end='')
        print_list(memory_history)
        print('Input : ', end='')
        print_list(input_history)
        print('Gate  : ', end='')
        print_list(gate_history)
        total_loss, per_position_loss = loss(correct[1:], result[1:])
        loss_str = 'Loss: **{:.5f}** = <span style="font-family:monaco">'.format(total_loss)
        for idx, per_pos_loss in enumerate(per_position_loss):
            loss_str += '{}{:.3f}'.format(' + ' if idx > 0 else '', per_pos_loss)
        loss_str += '</span>'
        printmd(loss_str)
        w1 -= alpha * dLdw1_gated(test_seq, correct, result, memory_history, input_history, gate_history)
        w2 -= alpha * dLdw2_gated(test_seq, correct, result, memory_history, input_history, gate_history)
        if not fixed_w3:
            w3 -= alpha * dLdw3(test_seq, correct, result, memory_history)
        w4 -= alpha * dLdw4_gated(test_seq, correct, result, memory_history, input_history, gate_history)
        w5 -= alpha * dLdw5_gated(test_seq, correct, result, memory_history, input_history, gate_history)
        print('w1={:+.3f}, w2={:+.3f}, w3={:+.3f}, w4={:+.3f}, w5={:+.3f}'.format(w1, w2, w3, w4, w5))
        print()
        reset_gated()
    return w1, w2, w3, w4, w5

In [15]:
experiment_gated('ab(ab)ab', _w1=1.0, _w2=1.0, _w4=1.0, _w5=1.0, alpha=5e-2, max_iter=250, fixed_w3=False)

w1=+1.000, w2=+1.000, w3=+1.000, w4=+1.000, w5=+1.000


**Iteration 0**

Output: [+0.000, +0.853, +0.821, +0.905, +0.982, +0.978, +0.989, +0.998, +0.998]
Target: [+0.000, +0.500, +0.500, +0.731, +0.731, +0.731, +0.500, +0.500, +0.500]
Memory: [+0.000, +1.762, +1.523, +2.254, +4.016, +3.777, +4.508, +6.270, +6.032]
Input : [+0.000, +2.000, -2.000, +1.000, +2.000, -2.000, +1.000, +2.000, -2.000]
Gate  : [+0.000, +0.881, +0.119, +0.731, +0.881, +0.119, +0.731, +0.881, +0.119]


Loss: **8.04865** = <span style="font-family:monaco">0.346 + 0.266 + 0.124 + 0.516 + 0.456 + 1.572 + 2.444 + 2.325</span>

<span style="font-family:monaco; font-size:12px">dL/dw1 = <span style="color:blue">+0.311</span> + <span style="color:blue">+0.244</span> + <span style="color:blue">+0.260</span> + <span style="color:blue">+0.596</span> + <span style="color:blue">+0.556</span> + <span style="color:blue">+1.103</span> + <span style="color:blue">+1.562</span> + <span style="color:blue">+1.501</span> = <span style="color:blue; text-decoration:underline">+6.132</span></span>

...

<span style="font-family:monaco; font-size:12px">dL/dw3 = <span style="color:blue">+0.032</span> + <span style="color:blue">+0.023</span> + <span style="color:red">-0.269</span> + <span style="color:red">-0.377</span> + <span style="color:red">-0.364</span> + <span style="color:blue">+0.202</span> + <span style="color:blue">+0.392</span> + <span style="color:blue">+0.358</span> = <span style="color:red; text-decoration:underline">-0.003</span></span>

<span style="font-family:monaco; font-size:12px">dL/dw4 = <span style="color:blue">+0.001</span> + <span style="color:blue">+0.001</span> + <span style="color:red">-0.010</span> + <span style="color:red">-0.011</span> + <span style="color:red">-0.014</span> + <span style="color:blue">+0.007</span> + <span style="color:blue">+0.012</span> + <span style="color:blue">+0.013</span> = <span style="color:red; text-decoration:underline">-0.002</span></span>

<span style="font-family:monaco; font-size:12px">dL/dw5 = <span style="color:blue">+0.001</span> + <span style="color:blue">+0.001</span> + <span style="color:red">-0.006</span> + <span style="color:red">-0.008</span> + <span style="color:red">-0.011</span> + <span style="color:blue">+0.006</span> + <span style="color:blue">+0.011</span> + <span style="color:blue">+0.012</span> = <span style="color:blue; text-decoration:underline">+0.005</span></span>

w1=+0.836, w2=+0.327, w3=+0.128, w4=+0.968, w5=+0.881



(0.8355377299370825,
 0.32694496106464,
 0.12751073601327018,
 0.968230391392321,
 0.8810238598477301)